In [35]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt

PREFIX='a video of '

def check_precision(frames_number, video_name, predicted_events, event, anomaly_classes, prompts, mode):
    # TN Prediction and Reality are true
    # TN Prediction and Reality are false
    # FN Prediction is false and Reality is true
    # FP Prediction is true and Reality is false
    normal_class = PREFIX + "a normal view (persons walking or standing)"
    all_classes = [normal_class] + anomaly_classes
    frames = np.load("/home/ubuntu/Database/ALL/GT/gt_ALL.npz")
    frames= frames[video_name]
    frames = np.append(frames, frames[-1])
    # Create a dictionary to convert class names to numeric indices
    class_to_idx = {cls: idx for idx, cls in enumerate(all_classes)}
    # Example: {"a normal view...": 0, "a person riding...": 1, ...}
    num_classes = len(all_classes)  # 14
    cm = np.zeros((num_classes, num_classes), dtype=int)
    prompts = [prompt.lower().split('.')[0] for prompt in prompts]
    # Save frames_number, predicted_events, and prompts into a numpy array
    fn_stored=0
    tn_stored=0
    for i in range(len(predicted_events)):
        # Get ground truth
        is_anomaly = frames[frames_number[i] - 1]  # 0 or 1
        # Determine true class using EVENT when anomaly exists
        true_class = event if is_anomaly == 1 else normal_class
        pred_class = predicted_events[i]
        if prompts[i] == "" and (mode==0 or mode==2):
            pass
            '''elif pred_class==normal_class:
            if is_anomaly == 0:
                tn_stored += 1
            else:
                fn_stored += 1
            continue'''
        elif prompts[i] == "yes":
            pass
        elif prompts[i] == "no":
            continue
        else:
            continue
        # Get predicted class
        
        # Convert to indices (skip if class not recognized)
        true_idx = class_to_idx.get(true_class, -1)
        pred_idx = class_to_idx.get(pred_class, -1)
        
        if true_idx != -1 and pred_idx != -1:
            cm[true_idx, pred_idx] += 1
    # Get the index of your event class
    event_idx = class_to_idx[event]
    

    # Calculate metrics ONLY for your event class
    tp = cm[event_idx, event_idx]  # TNs for event
    fp = np.sum(cm[:, event_idx]) - tp  # FPs (other classes predicted as event)
    fn =  np.sum(cm[event_idx, :]) - tp  # FNs (event misclassified as others)
    tn = np.sum(cm) - tp - fp - fn  # TNs
    fn += fn_stored
    tn += tn_stored
    return tp, fp, fn, tn
events = [
        "Riding",
        "Fighting",
        "Playing",
        "Running",
        'Lying',
        "Chasing",
        "Jumping",
        "Falling",
        "Guiding",
        "Stealing",
        "Littering",
        "Tripping",
        "Pickpockering",
    ]
description = [
        "a person riding a bicycle on the street",  # Added context
        "multiple people engaged in a physical fight",  # More specific than "fighting"
        "a group of people playing a sport together",  # Added "sport" for visual clarity
        "a person running",  # Added context
        "a person lying motionless on the ground",  # "Motionless" helps distinguish from falling
        "a person aggressively chasing another person",  # "Aggressively" adds distinction
        "a person jumping high in the air with both feet",  # More specific than just "jumping"
        "a person accidentally falling to the ground",  # "Accidentally" helps distinguish
        "a person gently guiding another person by the arm",  # Added detail
        "a person stealing other person",  # More specific than "stealing"
        "a person deliberately throwing garbage on the ground",  # "Deliberately" adds clarity
        "a person tripping over an obstacle",  # More descriptive
        "a person pickpocketing a wallet from someone's pocket",  # Very specific
    ]
description=[PREFIX+desc for desc in description]
events_description = {event: desc for event, desc in zip(events, description)}

In [36]:
rute="/home/ubuntu/Database/ALL/Videos"
rute_stored='/home/ubuntu/Tesis/Storage/'
modes=[0,1]
modes=[2]
usage='CLIP'
i=0
df= pd.DataFrame(columns=['Name', 'Event', 'Mode', 'TP', 'FP', 'FN', 'TN'])
for mode in modes:
    for folder_number in range(len(events)):
        folder=f"{rute}/{events[folder_number]}/"
        files= os.listdir(folder)
        for file in files:
            stored_file=file.split('.')[0]
            desc=events_description[events[folder_number]]
            info_stored=np.load(f"{rute_stored}{stored_file}_{usage}_{mode}_{desc}.npy", allow_pickle=True)
            frames_number=info_stored[0]
            predicted_events=info_stored[1]
            prompts=info_stored[2]
            event= events_description[events[folder_number]]
            tp, fp,fn, tn=check_precision(frames_number, stored_file, predicted_events, event, description, prompts, mode)
            row = {
            'Name':stored_file, 'Event':event, 'Mode':mode, 'TP':tp, 'FP':fp, 'FN':fn, 'TN':tn
            }
            df =pd.concat([df, pd.DataFrame([row])], ignore_index=True)
print(df)

        Name                                              Event Mode  TP FP  \
0     000350  a video of a person riding a bicycle on the st...    2   0  0   
1     000289  a video of a person riding a bicycle on the st...    2  25  0   
2    4_076_1  a video of a person riding a bicycle on the st...    2   9  0   
3     000326  a video of a person riding a bicycle on the st...    2   0  0   
4     000313  a video of a person riding a bicycle on the st...    2   0  0   
..       ...                                                ...  ...  .. ..   
247  1_075_1  a video of a person pickpocketing a wallet fro...    2   0  0   
248  4_086_1  a video of a person pickpocketing a wallet fro...    2   0  0   
249  D158_13  a video of a person pickpocketing a wallet fro...    2   0  0   
250  D149_07  a video of a person pickpocketing a wallet fro...    2   0  0   
251  D149_03  a video of a person pickpocketing a wallet fro...    2   0  0   

      FN   TN  
0     93   46  
1     81  137  
2  

In [37]:
df_modify=pd.read_csv('/home/ubuntu/Tesis/Results/TestingDevOnlyCLIP.csv')
df.rename(columns={'TP': 'True Positive', 'TN':'True Negative', 'FN':'False Negative', 'FP': 'False Positive'}, inplace=True)
for index, row in df.iterrows():
    # Find the matching row in df_modify
    match = df_modify[(df_modify['Name'].str.split('.').str[0] == row['Name']) & 
                    (df_modify['True Event'] == row['Event']) & 
                    (df_modify['Mode'] == int(row['Mode']))]
    if not match.empty:
        # Update the values in df_modify
        df_modify.loc[match.index, 'True Positive'] = int(row['True Positive'])
        df_modify.loc[match.index, 'False Positive'] = int(row['False Positive'])
        df_modify.loc[match.index, 'False Negative'] = int(row['False Negative'])
        df_modify.loc[match.index, 'True Negative'] = int(row['True Negative'])

In [38]:
df_modify.to_csv('/home/ubuntu/Tesis/Results/Tesis/Best_CLIP/TestingOnlyCLIP.csv', index=False)
print(df_modify)

            Name  Mode  True Positive  False Positive  False Negative  \
0     000350.avi     2              0               0              93   
1     000289.avi     2             25               0              81   
2    4_076_1.mp4     2              9               0              39   
3     000326.avi     2              0               0              53   
4     000313.avi     2              0               0             101   
..           ...   ...            ...             ...             ...   
247  1_075_1.mp4     2              0               0              20   
248  4_086_1.mp4     2              0               0              45   
249  D158_13.avi     2              0               0              20   
250  D149_07.avi     2              0               0              24   
251  D149_03.avi     2              0               0              78   

     True Negative                                         True Event  \
0               46  a video of a person riding a b